1\. Write a function that converts number representation (bin<->dec<->hex)

In [15]:
def converter(n, to):
    if to == "bin": # convert to binary
        return bin(n)
    elif to == "dec": # convert to decimal 
        return int(n)
    elif to == "hex": # convert to hexadecimal
        return hex(n)
    
print(converter(12, "bin"))
print(converter(12, "hex"))
print(converter(0x18, "dec"))
print(converter(0b0100, "dec"))
print(converter(0x18, "bin"))

0b1100
0xc
24
4
0b11000


2. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [8]:
# check input of 32 
def is32long(value):      
    if len(value) == 32:
            return True
    return False
    
def binaryToInt(value):
    bias = 127
    sign_bit = int(value[0])
    exponent = int(value[1:9], 2) - bias
    mantissa = value[9:]
    mantissa_int = 0
    exp_2 = -1
    for i in mantissa:
        mantissa_int += (int(i) * (2**exp_2))
        exp_2 -= 1

    return ((-1)**sign_bit)*(mantissa_int+1)*(2**exponent) # calculating the conversion
    
# input of a 32 bit word
bit32 = input()
if is32long(bit32) is True:
    print(binaryToInt(bit32)) # calling the function for conversion
else:
    print("Wrong length")
        

00000011111000000000000000000000
1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [75]:
import sys

underflow = 1
overflow = 1

j = 0
for i in range (1074) :
    underflow = underflow / 2
    j += 1
    
print("Underflow occurs in " + str(j) + " iterations with value " + str(underflow) + "\n")

print("Integers in python3 are represented with arbitrary precision and I can not have an overflow, at least in an acceptable time.")
print("The overflow definition can be applied to the architecture, in this case, with unsigned word = " + str(sys.maxsize*2 + 1))


Underflow occurs in 1074 iterations with value 5e-324

Integers in python3 are represented with arbitrary precision and I can not have an overflow, at least in an acceptable time.
The overflow definition can be applied to the architecture, in this case, with unsigned word = 18446744073709551615


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [6]:
var = 2
tmp = var

exp = -1
added = 10
count = 0
while count != 324:
    if (added**exp) == 0.0:
        break
    tmp = tmp + added**exp
    exp -= 1
    count += 1
print("The maximum precision is: " + str(added**(exp+1)))


The maximum precision is: 1e-323


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [67]:
import math

# a
def quad_equation(a, b, c):
    sol = [(-b + ((b**2-4*a*c)**(1/2))) / (2*a), (-b - ((b**2-4*a*c)**(1/2)))/ (2*a)]
    return sol

print(quad_equation(0.001, 1000, 0.001))

# b
def quad_equation_re(a, b, c):
    sol = [((-b + math.sqrt(b**2-4*a*c))*(-b - math.sqrt(b**2-4*a*c))) / ((2*a)*(-b - math.sqrt(b**2-4*a*c))), 
           ((-b - math.sqrt(b**2-4*a*c))*(-b + math.sqrt(b**2-4*a*c))) / ((2*a)*(-b + math.sqrt(b**2-4*a*c)))]
    return sol

print(quad_equation_re(0.001, 1000, 0.001))

# c
def roots(a, b, c):
    sol = [(2*c)/(-b - ((b**2 -4*a*c)**(1/2))), (-b-(b**2 -4*a*c)**(1/2))/(2*a)]
    return sol
print(roots(0.001, 1000, 0.001))

[-9.999894245993346e-07, -999999.999999]
[-9.999894245993346e-07, -999999.9999990001]
[-1.000000000001e-06, -999999.999999]


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [72]:
# function  𝑓(𝑥)=𝑥(𝑥−1)
def func(x):
    return x*(x-1)

def derivative(x, delta):
    f = func(x)
    f_delta = func(x+delta)
    return (f_delta-f)/delta
    

# a
x = 1
delta = 10**(-2)

res = derivative(x, delta)
print("The restul of the derivative is = " + str(res))
print("The results are not perfectly matched due to the mathematical representation with infinitesimal calculation caused by the computational limits\n")

# b
delta_list = [1e-4, 1e-6, 1e-8, 1e-10, 1e-12, 1e-14]
for d in delta_list:
    print(derivative(x, d))
print("\nFor the first three values there is linear decrease, while after it has a increase")
print("With the last value we have a different result approximate down")

The restul of the derivative is = 1.010000000000001
The results are not perfectly matched due to the mathematical representation with infinitesimal calculation caused by the computational limits

1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509

For the first three values there is linear decrease, while after it has a increase
With the last value we have a different result approximate down


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [68]:
from math import sqrt, pi
import time

# a Riemann definition of integral
def riemann_integral(N):
    h = 2/N
    summing = 0
    for k in range(N):
        summing += math.sqrt(1 - (-1+(h*k))**2)
        
    return h*summing

N = 100
found = riemann_integral(N)
true_value = pi/2
print("The riemann definition with N = 100 : " + str(found))
print("Compared to the true value it is lower then it about by a 0.001: " + str(true_value-found))
print("Accuracy: " + str(found/true_value))


# b find max N for max 1 second of computation
starting = time.time()
value = riemann_integral(4500000)
diff = time.time() - starting
print("\nTime elapsed = " + str(diff))
print("Value found in this time = " + str(value))
print("Compared to the true value it is lower then it about by a 0.001: " + str(true_value-value))
print("Accuracy: " + str(value/true_value))

# with 1 minute
starting = time.time()
value = riemann_integral(400000000)
diff = time.time() - starting
print("\nTime elapsed = " + str(diff))
print("Value found in this time = " + str(value))
print("Compared to the true value it is lower then it about by a 0.001: " + str(true_value-value))
print("Accuracy: " + str(value/true_value))
print("We can have more accuracy with more \"slices\"")


   

The riemann definition with N = 100 : 1.5691342555492498
Compared to the true value it is lower then it about by a 0.001: 0.001662071245646768
Accuracy: 0.9989418925819377

Time elapsed = 0.9453005790710449
Value found in this time = 1.5707963266207092
Compared to the true value it is lower then it about by a 0.001: 1.741873312255393e-10
Accuracy: 0.9999999998891089

Time elapsed = 84.08630084991455
Value found in this time = 1.5707963267954925
Compared to the true value it is lower then it about by a 0.001: -5.95967719618784e-13
Accuracy: 1.0000000000003795
We can have more accuracy with more "slices"
